In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
airbnb=pd.read_csv('airbnb_data.csv')
airbnb2=pd.read_csv('airbnb_submission.csv')
airbnb3=pd.read_csv('submit_example.csv')

In [5]:
#Outliers
airbnb["removed_outliers"] = airbnb.price.between(airbnb.price.quantile(.015), airbnb.price.quantile(.99))
outliers = airbnb[airbnb['removed_outliers'] == False]
airbnb=airbnb[airbnb['removed_outliers'] == True]
airbnb.drop(columns=["removed_outliers"],inplace=True)   

In [6]:
#columna id
id1=airbnb2.id
airbnb2.drop(columns=["id"],inplace=True)    

# Limpieza

Hacer un Dataframe con datos numericos, uno con fechas y otro con dummies    


In [7]:
class limpieza:
    
    def __init__(self,data):
        self.data = data
        print("<----------------DataFrame a Transformar --------------->",data.info())
        self.reescalado(data)
    
    def reescalado(self, data):
        def escala(x):
            return (x/100)
        def escalastr(x):
            if '%' in x:
                r = float(x.replace('%', ''))/100   
            else:
                r=x
            return float(r)
        def tf(x):
            if "f" in x:
                r=0
            if "t" in x:
                r=1
            return int(r)
        
        data["host_response_rate"]=data["host_response_rate"].apply(escalastr)
        data["instant_bookable"]=data["instant_bookable"].apply(tf)
        data["review_scores_rating"]=data["review_scores_rating"].apply(escala)
        data["host_has_profile_pic"]=data["host_has_profile_pic"].apply(tf)
        data["host_identity_verified"]=data["host_identity_verified"].apply(tf)
        self.dummies(data)
        
    def dummies(self,data):
        df1=pd.get_dummies(data.property_type,drop_first=False)
        dfpropetytype=df1[['Camper/RV', 'Dorm', 'Hostel', 'Tent', 'Yurt']]
        dfroomtype=pd.get_dummies(data.room_type,drop_first=False)
        dfcancellation=pd.get_dummies(data.cancellation_policy,drop_first=False).drop(columns=["super_strict_30","super_strict_60"])
        dfcity=pd.get_dummies(data.city,drop_first=False)
        dfbed_type=pd.get_dummies(data.bed_type,drop_first=False)
        airbnbdummies=pd.concat([dfpropetytype, dfroomtype,dfcancellation,dfcity,dfbed_type], axis=1)
        self.drop(data)
        self.output1(airbnbdummies)
        
    def drop(self,data):
        data.drop(columns=['description'],inplace=True)
        data.drop(columns=['name','neighbourhood','thumbnail_url'],inplace=True)
        data.drop(columns=['zipcode','room_type','cancellation_policy'],inplace=True)
        data.drop(columns=['city','amenities','bed_type'],inplace=True)              
        data.drop(columns=['property_type'],inplace=True)  
        self.cambiotype(data)
        
    def cambiotype(self,data):
        
        for e in data.select_dtypes('object').columns:
            data[e]=data[e].astype('category')
        for e in data.select_dtypes('integer').columns:
            data[e]=pd.to_numeric(data[e], downcast='integer')
        for e in data.select_dtypes('float').columns:
            data[e]=pd.to_numeric(data[e], downcast='float')
        data["bedrooms"]=data.bedrooms.astype(int)
        data["beds"]=data.beds.astype(int)
        data["cleaning_fee"]=data.cleaning_fee.astype(int)
        data["first_review"]=pd.to_datetime(data.first_review) 
        data["host_since"]=pd.to_datetime(data.host_since) 
        data["last_review"]=pd.to_datetime(data.last_review) 
        self.dates(data)
        self.output(data)
        
    def dates(self,data):
        
        airbnbdate=data[["first_review","host_since","last_review"]]
        data.drop(columns=["first_review","host_since","last_review"],inplace=True) 
        self.output2(airbnbdate)
        
    def output(self,data):
        print("<----------------DataFrame a Limpio --------------->",data.info())
        global Dataclean
        Dataclean=data
   
    def output1(self,dummies):
        self.dummies = dummies
        print("<----------------DataFrame a de Dummies --------------->",dummies.info())
        global Datadummies
        Datadummies=dummies
    
    def output2(self,dates):
        self.dates = dates
        print("<----------------DataFrame de Dates --------------->",dates.info())
        global Datadates
        Datadates=dates


In [8]:
limpieza(airbnb)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28188 entries, 0 to 28876
Data columns (total 28 columns):
property_type             28188 non-null object
room_type                 28188 non-null object
amenities                 28188 non-null object
accommodates              28188 non-null int64
bathrooms                 28188 non-null float64
bed_type                  28188 non-null object
cancellation_policy       28188 non-null object
cleaning_fee              28188 non-null bool
city                      28188 non-null object
description               28188 non-null object
first_review              28188 non-null object
host_has_profile_pic      28188 non-null object
host_identity_verified    28188 non-null object
host_response_rate        28188 non-null object
host_since                28188 non-null object
instant_bookable          28188 non-null object
last_review               28188 non-null object
latitude                  28188 non-null float64
longitude                 28

In [9]:
#separo columna target
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter('ignore')
y=Dataclean.price
Dataclean.drop(columns=["price"],inplace=True) 

In [11]:
#Datasets diferentes limpios

X=Dataclean
XD=Datadummies
XT=pd.concat([Dataclean,Datadummies], axis=1)

In [13]:
#Normalizacion 

Xnormal=StandardScaler().fit_transform(X)
XDnormal=StandardScaler().fit_transform(XD)
XTnormal=StandardScaler().fit_transform(XT)

In [ ]:
#Dataset 2

In [14]:
limpieza(airbnb2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9625 entries, 0 to 9624
Data columns (total 27 columns):
property_type             9625 non-null object
room_type                 9625 non-null object
amenities                 9625 non-null object
accommodates              9625 non-null int64
bathrooms                 9625 non-null float64
bed_type                  9625 non-null object
cancellation_policy       9625 non-null object
cleaning_fee              9625 non-null bool
city                      9625 non-null object
description               9625 non-null object
first_review              9625 non-null object
host_has_profile_pic      9625 non-null object
host_identity_verified    9625 non-null object
host_response_rate        9625 non-null object
host_since                9625 non-null object
instant_bookable          9625 non-null object
last_review               9625 non-null object
latitude                  9625 non-null float64
longitude                 9625 non-null float64


In [15]:
X2=Dataclean
XD2=Datadummies
XT2=pd.concat([Dataclean,Datadummies], axis=1)
XT2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9625 entries, 0 to 9624
Data columns (total 35 columns):
accommodates              9625 non-null int8
bathrooms                 9625 non-null float32
cleaning_fee              9625 non-null int32
host_has_profile_pic      9625 non-null int8
host_identity_verified    9625 non-null int8
host_response_rate        9625 non-null float32
instant_bookable          9625 non-null int8
latitude                  9625 non-null float32
longitude                 9625 non-null float32
number_of_reviews         9625 non-null int16
review_scores_rating      9625 non-null float32
bedrooms                  9625 non-null int32
beds                      9625 non-null int32
Camper/RV                 9625 non-null uint8
Dorm                      9625 non-null uint8
Hostel                    9625 non-null uint8
Tent                      9625 non-null uint8
Yurt                      9625 non-null uint8
Entire home/apt           9625 non-null uint8
Private room 

In [16]:
Xnormal2=StandardScaler().fit_transform(X2)
XDnormal2=StandardScaler().fit_transform(XD2)
XTnormal2=StandardScaler().fit_transform(XT2)

# Modelos

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor as SGDR
from sklearn.neighbors import KNeighborsRegressor as KNNR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR
import time

In [18]:
#listas donde se agregan predicciones de araña 1 y 2
dt=[]
ytest=[]
yfinal=[]

In [19]:
class modelar:
    
    def __init__(self,X,y,train=None):
        self.X = X
        self.y = y
        self.train=train
        self.split(X,y,train)
        
    def split(self,X,y,train):
        
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)
        self.regresion(X_train, X_test, y_train, y_test)
        self.randomforest(X_train, X_test, y_train, y_test,train)
        self.tree(X_train, X_test, y_train, y_test)
        self.lasso(X_train, X_test, y_train, y_test)
        self.ridge(X_train, X_test, y_train, y_test,train)
        self.svr(X_train, X_test, y_train, y_test)
        self.sgdr(X_train, X_test, y_train, y_test)
        time.sleep(2)
        self.neighbors(X_train, X_test, y_train, y_test)
        self.GradientBoost(X_train, X_test, y_train, y_test,train)
        time.sleep(2)
        self.XBoost(X_train, X_test, y_train, y_test,train)
        time.sleep(2)
        self.CatBoost(X_train, X_test, y_train, y_test,train)
        time.sleep(2)
        self.ligth(X_train, X_test, y_train, y_test,train)
        
    def regresion(self,X_train,X_test,y_train,y_test):
        ytest.extend(y_test)
        linreg=LinearRegression(fit_intercept=True)
        linreg.fit(X_train, y_train)
        y_pred=linreg.predict(X_test)
        print("El rmse de regresion lineal es:",mse(y_test, y_pred)**(0.5))
        
    def randomforest(self,X_train,X_test,y_train,y_test,train):
        rf=RFR()
        rf.fit(X_train, y_train)
        y_pred=rf.predict(X_test)
        dt.append(y_pred)
        yf_pred=rf.predict(train)
        yfinal.append(yf_pred)
        print("El rmse de RandomForest es:",mse(y_test, y_pred)**(0.5))
        
    def tree (self,X_train,X_test,y_train,y_test):
        etr=ETR(min_samples_leaf=2)
        etr.fit(X_train, y_train)
        y_pred=etr.predict(X_test)
        print("El rmse de tree es:",mse(y_test, y_pred)**(0.5))
        
    def lasso (self,X_train,X_test,y_train,y_test):
        lasso=Lasso()
        lasso.fit(X_train, y_train)
        y_pred=lasso.predict(X_test)
        print("El rmse de regresion lasso es:",mse(y_test, y_pred)**(0.5))
        
    def ridge (self,X_train,X_test,y_train,y_test,train):
        ridge=Ridge()
        ridge.fit(X_train, y_train)
        y_pred=ridge.predict(X_test)
        dt.append(y_pred)
        yf_pred=ridge.predict(train)
        print("El rmse de regresion ridge es:",mse(y_test, y_pred)**(0.5))
        
    def svr (self,X_train,X_test,y_train,y_test):
        svr=SVR()
        svr.fit(X_train, y_train)
        y_pred=svr.predict(X_test)
        print("El rmse de SVR es:",mse(y_test, y_pred)**(0.5))
        
    def sgdr (self,X_train,X_test,y_train,y_test):
        sgdr=SGDR()
        sgdr.fit(X_train, y_train)
        y_pred=sgdr.predict(X_test)
        print("El rmse de SGDR es:",mse(y_test, y_pred)**(0.5))
        
    def neighbors (self,X_train,X_test,y_train,y_test):
        knnr=KNNR(n_neighbors=7)
        knnr.fit(X_train, y_train)
        y_pred=knnr.predict(X_test)
        print("El rmse de neighbors es:",mse(y_test, y_pred)**(0.5))
        
    def GradientBoost (self,X_train,X_test,y_train,y_test,train):
        gbr=GBR()
        gbr.fit(X_train, y_train)
        y_pred=gbr.predict(X_test)
        dt.append(y_pred)
        yf_pred=gbr.predict(train)
        yfinal.append(yf_pred)
        print("El rmse de GradientBoost es:",mse(y_test, y_pred)**(0.5))
        
    def XBoost (self,X_train,X_test,y_train,y_test,train):
        xgbr=XGBR(learning_rate= 0.22709644562829365, n_estimators= int(150.0), x_gamma= 0.24233002080834415, 
                  x_max_depth= int(10.0), x_min_child= 8.0, x_reg_lambda= 0.43851924417831933, x_subsample= 0.9685862989462508)
        xgbr.fit(X_train, y_train)
        y_pred=xgbr.predict(X_test)
        dt.append(y_pred)
        yf_pred=xgbr.predict(train)
        yfinal.append(yf_pred)
        yf_pred=xgbr.predict(train)
        yfinal.append(yf_pred)
        print("El rmse de XBoost es:",mse(y_test, y_pred)**(0.5))
        
    def CatBoost(self,X_train,X_test,y_train,y_test,train):
        ctr=CTR()
        ctr.fit(X_train, y_train)
        y_pred=ctr.predict(X_test)
        dt.append(y_pred)
        yf_pred=ctr.predict(train)
        yfinal.append(yf_pred)
        print("El rmse de CatBoost es:",mse(y_test, y_pred)**(0.5))
        
    def ligth (self,X_train,X_test,y_train,y_test,train):
        lgbmr=LGBMR()
        lgbmr.fit(X_train, y_train)
        y_pred=lgbmr.predict(X_test)
        dt.append(y_pred)
        yf_pred=lgbmr.predict(train)
        yfinal.append(yf_pred)
        print("El rmse de ligth es:",mse(y_test, y_pred)**(0.5))
        
    
        
    

In [ ]:
#hacer que la araña acepte 3 variables y ver la tendencia general de desvacion un epsilon ver suma de diferencias limpiar outliers

In [ ]:
modelar(XTnormal,y,XTnormal2)

In [ ]:
dt
dtDf= pd.DataFrame(dt)  
datay= dtDf.T
datay.drop(columns=[1],inplace=True)  
datay["mean"]=datay.mean(numeric_only=True, axis=1)
datay["ytest"]=ytest
datay["diff"]=datay["mean"]-datay["ytest"]
datay["diff"].sum()
datay["mean2"]=datay["mean"]*.9974
datay["diff2"]=datay["mean2"]-datay["ytest"]
datay["diff2"].sum()


In [ ]:
ytest

In [ ]:
yfinal
dtFinal= pd.DataFrame(yfinal)  
datayfinal= dtFinal.T
datayfinal["mean"]=datayfinal.mean(numeric_only=True, axis=1)
datayfinal["mean2"]=datayfinal["mean"]*.9974
datayfinal.head()
 

In [ ]:
datayfinal

"Test con primer dataset normal(solo datos numericos)"
El rmse de regresion lineal es: 100.6172059514566
El rmse de regresion lineal es: 98.64658361593311
El rmse de regresion lineal es: 108.41626715575023
El rmse de regresion lineal es: 100.7099897054601
El rmse de regresion lineal es: 100.61720486349266
El rmse de regresion lineal es: 113.6531603018469
El rmse de regresion lineal es: 101.03838495417904
El rmse de regresion lineal es: 95.66117575524044
El rmse de regresion lineal es: 90.531534930734
El rmse de regresion lineal es: 90.18695317748293
El rmse de regresion lineal es: 89.3204033769017
El rmse de regresion lineal es: 90.71653283743059

"Test con segundo dataset normalizado(solo datos categoricos en dummies)"
El rmse de regresion lineal es: 115.35641061787302
El rmse de regresion lineal es: 114.98445654299614
El rmse de regresion lineal es: 115.11248536683252
El rmse de regresion lineal es: 115.60177645304822
El rmse de regresion lineal es: 115.36394741875793
El rmse de regresion lineal es: 119.37954625880297
El rmse de regresion lineal es: 1789127.98820823
El rmse de regresion lineal es: 126.70975230077991
El rmse de regresion lineal es: 115.10368718712297
El rmse de regresion lineal es: 115.00917082998208
El rmse de regresion lineal es: 115.05355227853528
El rmse de regresion lineal es: 114.93007747644249

"Test con dataset Completo"
El rmse de regresion lineal es: 96.45012039558841
El rmse de RandomForest es: 93.26682029194988 <---
El rmse de tree es: 108.79175672920208
El rmse de regresion lasso es: 96.57809521379946
El rmse de regresion ridge es: 96.45139765494875 <---
El rmse de SVR es: 111.22965209843674
El rmse de SGDR es: 205777099.54264945
El rmse de neighbors es: 94.24569473754491
El rmse de GradientBoost es: 87.00181556412473 <---
El rmse de XBoost es: 86.88639201386773 <---
El rmse de CatBoost es: 86.72321009819711 <---
El rmse de ligth es: 87.38401680904394 <---


# Optimizacion

In [ ]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from xgboost import XGBRegressor as xgb

In [ ]:

X=XTnormal
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [ ]:
space={
    'n_estimators':hp.quniform('n_estimators', 10, 1000, 25),
    'learning_rate':hp.uniform('learning_rate', 0.0001, 1.0),
    'max_depth':hp.quniform('x_max_depth', 4, 16, 1),
    'min_child_weight':hp.quniform('x_min_child', 1, 10, 1),
    'subsample':hp.uniform('x_subsample', 0.7, 1),
    'gamma':hp.uniform('x_gamma', 0.1, 0.5),
    'reg_lambda':hp.uniform('x_reg_lambda', 0, 1)
}

In [ ]:
def objetivo(space):
    
    modelo=xgb(
        n_estimators=int(space['n_estimators']),
        learning_rate=space['learning_rate'],
        max_depth=int(space['max_depth']),
        min_child_weight=space['min_child_weight'],
        subsample=space['subsample'],
        gamma=space['gamma'],
        reg_lambda=space['reg_lambda'],
        objective='reg:squarederror'
    )
    
    eval_set=[(X_train, y_train), (X_test, y_test)]
    
    modelo.fit(X_train, y_train, eval_set=eval_set, eval_metric='rmse', verbose=False)
    
    y_pred=modelo.predict(X_test)
    
    rmse=mse(y_test, y_pred)**0.5
    
    return {'loss':rmse, 'status':STATUS_OK}

In [ ]:
best=fmin(fn=objetivo, space=space, algo=tpe.suggest, max_evals=10, trials=Trials())

In [ ]:
best


In [ ]:

sub_file = pd.DataFrame(columns=["id","price"])
sub_file['price'] = datayfinal["mean2"]
sub_file["id"]= id1

sub_file.to_csv('finalelipson.csv', index = False)

In [ ]:
sub_file.head()

In [20]:
import pandas as pd
airbnb3=pd.read_csv('submit_example.csv')

In [21]:
dt=[]
ytest=[]
yfinal=[]

In [22]:
class modelar:
    
    def __init__(self,X,y,train=None):
        self.X = X
        self.y = y
        self.train=train
        self.split(X,y,train)
        
    def split(self,X,y,train):
        
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)
        self.regresion(X, y,train)
        self.randomforest(X, y,train)
        self.tree(X, y,train)
        self.lasso(X,y,train)
        self.ridge(X, y,train)
        self.svr(X, y,train)
        self.sgdr(X, y,train)
        time.sleep(2)
        self.neighbors(X, y,train)
        self.GradientBoost(X, y,train)
        time.sleep(2)
        self.XBoost(X,y,train)
        time.sleep(2)
        self.CatBoost(X,y,train)
        time.sleep(2)
        self.ligth(X,y,train)
        
    def regresion(self,X, y,train):
        ytest.append(y)
        linreg=LinearRegression(fit_intercept=True)
        linreg.fit(X, y)
        y_pred=linreg.predict(train)
        #print("El rmse de regresion lineal es:",mse(y_test, y_pred)**(0.5))
        
    def randomforest(self,X, y,train):
        rf=RFR()
        rf.fit(X, y)
        y_pred=rf.predict(train)
        dt.append(y_pred)
        yf_pred=rf.predict(train)
        yfinal.append(yf_pred)
        #print("El rmse de RandomForest es:",mse(y_test, y_pred)**(0.5))
        
    def tree (self,X, y,train):
        etr=ETR(min_samples_leaf=2)
        etr.fit(X, y)
        y_pred=etr.predict(train)
        #print("El rmse de tree es:",mse(y_test, y_pred)**(0.5))
        
    def lasso (self,X, y,train):
        lasso=Lasso()
        lasso.fit(X, y)
        y_pred=lasso.predict(train)
        #print("El rmse de regresion lasso es:",mse(y_test, y_pred)**(0.5))
        
    def ridge (self,X, y,train):
        ridge=Ridge()
        ridge.fit(X, y)
        y_pred=ridge.predict(train)
        dt.append(y_pred)
        yf_pred=ridge.predict(train)
        yfinal.append(yf_pred)
        #print("El rmse de regresion ridge es:",mse(y_test, y_pred)**(0.5))
        
    def svr (self,X, y,train):
        svr=SVR()
        svr.fit(X, y)
        y_pred=svr.predict(train)
        #print("El rmse de SVR es:",mse(y_test, y_pred)**(0.5))
        
    def sgdr (self,X, y,train):
        sgdr=SGDR()
        sgdr.fit(X, y)
        y_pred=sgdr.predict(train)
        #print("El rmse de SGDR es:",mse(y_test, y_pred)**(0.5))
        
    def neighbors (self,X, y,train):
        knnr=KNNR(n_neighbors=7)
        knnr.fit(X, y)
        y_pred=knnr.predict(train)
        #print("El rmse de neighbors es:",mse(y_test, y_pred)**(0.5))
        
    def GradientBoost (self,X, y,train):
        gbr=GBR()
        gbr.fit(X, y)
        y_pred=gbr.predict(train)
        dt.append(y_pred)
        yf_pred=gbr.predict(train)
        yfinal.append(yf_pred)
        #print("El rmse de GradientBoost es:",mse(y_test, y_pred)**(0.5))
        
    def XBoost (self,X, y,train):
        xgbr=XGBR(learning_rate= 0.22709644562829365, n_estimators= int(150.0), x_gamma= 0.24233002080834415, 
                  x_max_depth= int(10.0), x_min_child= 8.0, x_reg_lambda= 0.43851924417831933, x_subsample= 0.9685862989462508)
        xgbr.fit(X, y)
        y_pred=xgbr.predict(train)
        dt.append(y_pred)
        yf_pred=xgbr.predict(train)
        #yfinal.append(yf_pred)
        #print("El rmse de XBoost es:",mse(y_test, y_pred)**(0.5))
        
    def CatBoost(self,X, y,train):
        ctr=CTR()
        ctr.fit(X, y)
        y_pred=ctr.predict(train)
        dt.append(y_pred)
        yf_pred=ctr.predict(train)
        yfinal.append(yf_pred)
        #print("El rmse de CatBoost es:",mse(y_test, y_pred)**(0.5))
        
    def ligth (self,X, y,train):
        lgbmr=LGBMR()
        lgbmr.fit(X, y)
        y_pred=lgbmr.predict(train)
        dt.append(y_pred)
        yf_pred=lgbmr.predict(train)
        yfinal.append(yf_pred)
        #print("El rmse de ligth es:",mse(y_test, y_pred)**(0.5))
        

In [23]:
modelar(XTnormal,y,XTnormal2)

[20:44:55] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0:	learn: 97.4968563	total: 223ms	remaining: 3m 42s
1:	learn: 96.0438761	total: 237ms	remaining: 1m 58s
2:	learn: 94.5643546	total: 252ms	remaining: 1m 23s
3:	learn: 93.1738254	total: 266ms	remaining: 1m 6s
4:	learn: 91.8703900	total: 281ms	remaining: 55.8s
5:	learn: 90.7383693	total: 312ms	remaining: 51.7s
6:	learn: 89.5333284	total: 332ms	remaining: 47.1s
7:	learn: 88.4024331	total: 342ms	remaining: 42.5s
8:	learn: 87.3337815	total: 353ms	remaining: 38.9s
9:	learn: 86.2834290	total: 364ms	remaining: 36s
10:	learn: 85.2566341	total: 374ms	remaining: 33.6s
11:	learn: 84.2760677	total: 385ms	remaining: 31.7s
12:	learn: 83.3302188	total: 395ms	remaining: 30s
13:	learn: 82.4575847	total: 405ms	remaining: 28.5s
14:	learn: 81.6307664	total: 426ms	remaining: 28s
15:	learn: 80.7232186	total: 442ms	remaining: 27.2s
16:	learn: 79.927

161:	learn: 57.3410908	total: 2.24s	remaining: 11.6s
162:	learn: 57.3085076	total: 2.26s	remaining: 11.6s
163:	learn: 57.2694806	total: 2.28s	remaining: 11.6s
164:	learn: 57.2558665	total: 2.29s	remaining: 11.6s
165:	learn: 57.2306672	total: 2.3s	remaining: 11.6s
166:	learn: 57.2031551	total: 2.31s	remaining: 11.5s
167:	learn: 57.1771557	total: 2.32s	remaining: 11.5s
168:	learn: 57.1530676	total: 2.34s	remaining: 11.5s
169:	learn: 57.1379315	total: 2.35s	remaining: 11.5s
170:	learn: 57.1220040	total: 2.36s	remaining: 11.4s
171:	learn: 57.1099768	total: 2.38s	remaining: 11.4s
172:	learn: 57.0846088	total: 2.39s	remaining: 11.4s
173:	learn: 57.0568117	total: 2.4s	remaining: 11.4s
174:	learn: 57.0337542	total: 2.41s	remaining: 11.4s
175:	learn: 57.0013933	total: 2.43s	remaining: 11.4s
176:	learn: 56.9885917	total: 2.44s	remaining: 11.3s
177:	learn: 56.9779974	total: 2.45s	remaining: 11.3s
178:	learn: 56.9563898	total: 2.46s	remaining: 11.3s
179:	learn: 56.9306487	total: 2.48s	remaining: 1

324:	learn: 54.8364556	total: 4.56s	remaining: 9.48s
325:	learn: 54.8245950	total: 4.58s	remaining: 9.46s
326:	learn: 54.8156287	total: 4.59s	remaining: 9.44s
327:	learn: 54.8040570	total: 4.61s	remaining: 9.44s
328:	learn: 54.7886372	total: 4.62s	remaining: 9.43s
329:	learn: 54.7780593	total: 4.63s	remaining: 9.4s
330:	learn: 54.7674259	total: 4.64s	remaining: 9.38s
331:	learn: 54.7568561	total: 4.65s	remaining: 9.36s
332:	learn: 54.7426675	total: 4.66s	remaining: 9.34s
333:	learn: 54.7349583	total: 4.68s	remaining: 9.33s
334:	learn: 54.7152422	total: 4.69s	remaining: 9.32s
335:	learn: 54.7060847	total: 4.71s	remaining: 9.32s
336:	learn: 54.6913303	total: 4.73s	remaining: 9.3s
337:	learn: 54.6780166	total: 4.74s	remaining: 9.28s
338:	learn: 54.6687034	total: 4.75s	remaining: 9.27s
339:	learn: 54.6528586	total: 4.77s	remaining: 9.26s
340:	learn: 54.6463917	total: 4.78s	remaining: 9.25s
341:	learn: 54.6381831	total: 4.8s	remaining: 9.23s
342:	learn: 54.6241345	total: 4.81s	remaining: 9.

488:	learn: 53.2333849	total: 6.91s	remaining: 7.22s
489:	learn: 53.2263856	total: 6.93s	remaining: 7.21s
490:	learn: 53.2205006	total: 6.94s	remaining: 7.19s
491:	learn: 53.2125634	total: 6.95s	remaining: 7.18s
492:	learn: 53.2074527	total: 6.96s	remaining: 7.16s
493:	learn: 53.1925589	total: 6.97s	remaining: 7.14s
494:	learn: 53.1767422	total: 6.99s	remaining: 7.13s
495:	learn: 53.1668639	total: 7s	remaining: 7.11s
496:	learn: 53.1598726	total: 7.01s	remaining: 7.09s
497:	learn: 53.1548094	total: 7.02s	remaining: 7.08s
498:	learn: 53.1410592	total: 7.03s	remaining: 7.06s
499:	learn: 53.1342669	total: 7.04s	remaining: 7.04s
500:	learn: 53.1292362	total: 7.05s	remaining: 7.02s
501:	learn: 53.1217351	total: 7.06s	remaining: 7s
502:	learn: 53.1095064	total: 7.07s	remaining: 6.99s
503:	learn: 53.0963046	total: 7.08s	remaining: 6.97s
504:	learn: 53.0862450	total: 7.1s	remaining: 6.96s
505:	learn: 53.0777027	total: 7.11s	remaining: 6.94s
506:	learn: 53.0691796	total: 7.12s	remaining: 6.92s


644:	learn: 52.0869691	total: 8.7s	remaining: 4.79s
645:	learn: 52.0806758	total: 8.72s	remaining: 4.78s
646:	learn: 52.0744649	total: 8.74s	remaining: 4.77s
647:	learn: 52.0656539	total: 8.76s	remaining: 4.76s
648:	learn: 52.0544061	total: 8.78s	remaining: 4.75s
649:	learn: 52.0480087	total: 8.8s	remaining: 4.74s
650:	learn: 52.0421990	total: 8.81s	remaining: 4.72s
651:	learn: 52.0370568	total: 8.83s	remaining: 4.71s
652:	learn: 52.0320211	total: 8.84s	remaining: 4.7s
653:	learn: 52.0287728	total: 8.86s	remaining: 4.69s
654:	learn: 52.0246725	total: 8.88s	remaining: 4.68s
655:	learn: 52.0183239	total: 8.89s	remaining: 4.66s
656:	learn: 52.0134538	total: 8.91s	remaining: 4.65s
657:	learn: 52.0099508	total: 8.93s	remaining: 4.64s
658:	learn: 52.0036838	total: 8.95s	remaining: 4.63s
659:	learn: 51.9932418	total: 8.96s	remaining: 4.62s
660:	learn: 51.9878206	total: 8.97s	remaining: 4.6s
661:	learn: 51.9842888	total: 8.99s	remaining: 4.59s
662:	learn: 51.9807507	total: 9.01s	remaining: 4.5

804:	learn: 51.1359367	total: 11s	remaining: 2.67s
805:	learn: 51.1291359	total: 11.1s	remaining: 2.66s
806:	learn: 51.1232960	total: 11.1s	remaining: 2.65s
807:	learn: 51.1187606	total: 11.1s	remaining: 2.64s
808:	learn: 51.1098977	total: 11.1s	remaining: 2.62s
809:	learn: 51.1063156	total: 11.1s	remaining: 2.61s
810:	learn: 51.1046389	total: 11.2s	remaining: 2.6s
811:	learn: 51.0947698	total: 11.2s	remaining: 2.59s
812:	learn: 51.0853895	total: 11.2s	remaining: 2.58s
813:	learn: 51.0816167	total: 11.2s	remaining: 2.57s
814:	learn: 51.0763354	total: 11.3s	remaining: 2.56s
815:	learn: 51.0726350	total: 11.3s	remaining: 2.55s
816:	learn: 51.0654103	total: 11.3s	remaining: 2.53s
817:	learn: 51.0607833	total: 11.3s	remaining: 2.52s
818:	learn: 51.0554749	total: 11.3s	remaining: 2.51s
819:	learn: 51.0468629	total: 11.4s	remaining: 2.5s
820:	learn: 51.0427555	total: 11.4s	remaining: 2.49s
821:	learn: 51.0363664	total: 11.4s	remaining: 2.48s
822:	learn: 51.0286356	total: 11.4s	remaining: 2.4

970:	learn: 50.2717296	total: 13.6s	remaining: 406ms
971:	learn: 50.2673488	total: 13.6s	remaining: 393ms
972:	learn: 50.2621107	total: 13.6s	remaining: 379ms
973:	learn: 50.2585279	total: 13.7s	remaining: 364ms
974:	learn: 50.2554120	total: 13.7s	remaining: 350ms
975:	learn: 50.2515316	total: 13.7s	remaining: 336ms
976:	learn: 50.2493929	total: 13.7s	remaining: 322ms
977:	learn: 50.2444077	total: 13.7s	remaining: 308ms
978:	learn: 50.2396581	total: 13.7s	remaining: 294ms
979:	learn: 50.2350947	total: 13.7s	remaining: 280ms
980:	learn: 50.2318761	total: 13.7s	remaining: 266ms
981:	learn: 50.2293033	total: 13.7s	remaining: 252ms
982:	learn: 50.2249757	total: 13.7s	remaining: 238ms
983:	learn: 50.2225979	total: 13.8s	remaining: 224ms
984:	learn: 50.2136475	total: 13.8s	remaining: 210ms
985:	learn: 50.2088959	total: 13.8s	remaining: 196ms
986:	learn: 50.2033606	total: 13.8s	remaining: 182ms
987:	learn: 50.1968232	total: 13.8s	remaining: 168ms
988:	learn: 50.1945084	total: 13.8s	remaining:

In [24]:
dt
dtDf= pd.DataFrame(dt)  
datay= dtDf.T
datay["mean"]=datay.mean(numeric_only=True, axis=1)
display(datay.head(20))
airbnb3.head(20)

,0,1,2,3,4,5,mean
0,514.7,344.186540,478.556428,641.775574,602.027413,538.510605,519.959427
1,179.4,93.421977,141.717599,270.705902,183.110873,153.710921,170.344545
2,291.9,311.872978,274.661576,294.473572,255.294082,264.199921,282.067022
3,217.6,146.902564,153.864632,207.512985,215.360432,214.780021,192.670106
4,163.3,181.860219,168.132887,169.942062,167.668260,167.203342,169.684462
5,67.4,52.638515,78.386539,80.962288,75.983407,78.232454,72.267201
6,132.3,168.311206,138.385789,133.090683,139.312459,125.558001,139.493023
7,84.6,82.746410,57.752283,52.278702,61.897263,63.935060,67.201620
8,199.1,178.027482,190.824116,203.690735,201.763981,188.071836,193.579692
9,163.7,152.044263,163.175326,177.892059,202.870577,191.768534,175.241793


,id,price
0,0,95.911263
1,1,119.531982
2,2,91.904163
3,3,174.373548
4,4,169.217162
5,5,139.662500
6,6,386.133509
7,7,54.288577
8,8,173.292906
9,9,137.709449


In [25]:
yfinal
dtFinal= pd.DataFrame(yfinal)  
datayfinal= dtFinal.T
datayfinal["mean"]=datayfinal.mean(numeric_only=True, axis=1)
datayfinal.head()

,0,1,2,3,4,mean
0,514.7,344.186540,478.556428,602.027413,538.510605,495.596197
1,179.4,93.421977,141.717599,183.110873,153.710921,150.272274
2,291.9,311.872978,274.661576,255.294082,264.199921,279.585712
3,217.6,146.902564,153.864632,215.360432,214.780021,189.701530
4,163.3,181.860219,168.132887,167.668260,167.203342,169.632942


In [151]:
sub_file = pd.DataFrame(columns=["id","price"])
sub_file['price'] = datayfinal[2]
sub_file["id"]= id1
sub_file["price2"]=airbnb3["price"]
sub_file["price3"]=sub_file.price-sub_file.price2

sub_file["real"]= np.where(sub_file["price"] > sub_file["price2"], sub_file["price"]-(sub_file["price3"]*.9016), 
                                 sub_file["price"]+(abs(sub_file["price3"])*.90666))


sub_file.to_csv('realabsfinal.csv', index = False)
sub_file.head(20)

,id,price,price2,price3,real
0,0,478.556428,95.911263,382.645165,133.563547
1,1,141.717599,119.531982,22.185617,121.715047
2,2,274.661576,91.904163,182.757413,109.887492
3,3,153.864632,174.373548,-20.508916,172.459245
4,4,168.132887,169.217162,-1.084275,169.115956
5,5,78.386539,139.662500,-61.275961,133.943002
6,6,138.385789,386.133509,-247.747720,363.008736
7,7,57.752283,54.288577,3.463706,54.629406
8,8,190.824116,173.292906,17.531210,175.017977
9,9,163.175326,137.709449,25.465878,140.215291


In [153]:
sub = pd.DataFrame(columns=["id","price"])
sub["id"]= id1
sub["price"]=sub_file["real"]
sub.to_csv('elipsonabsfinalch.csv', index = False)

In [154]:
sub.head()

,id,price
0,0,133.563547
1,1,121.715047
2,2,109.887492
3,3,172.459245
4,4,169.115956
